In [ ]:
import torch
import torch.nn as nn
from PIL import Image
from torchvision import transforms
import os
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [ ]:
# Discriminator model

class Block(nn.Module):
  def __init__(self,in_channels,out_channels,stride):
    super().__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels,out_channels,4,stride,1,bias=True,padding_mode='reflect'),
        nn.InstanceNorm2d(out_channels),
        nn.ReLU(0.2),
    )

  def forward(self,x):
    return self.conv(x)

In [ ]:
# Discriminator model

class Discriminator(nn.Module):
  def __init__(self,in_channels = 3, features =[64,128,256,512]):
    super().__init__()
    self.initial= nn.Sequential(
        nn.Conv2d(in_channels,features[0],kernel_size=4,stride=2,padding=1,padding_mode='reflect'),
        nn.ReLU(0.2),
    )
    layers=[]
    in_channels=features[0]
    for feature in features[1:]:
      layers.append(Block(in_channels,feature,stride=1 if feature == features[-1] else 2))
      in_channels = feature
    layers.append(nn.Conv2d(in_channels,1,kernel_size=4,stride=1,padding=1,padding_mode='reflect'))
    self.model=nn.Sequential(*layers)

  def forward(self,x):
    x=self.initial(x)
    return torch.sigmoid(self.model(x))

In [ ]:
# Discriminator model

def test():
  x=torch.randn((1,3,256,256))
  model = Discriminator()
  preds=model(x)
  print(model)
  print(preds.shape)

In [ ]:
# Discriminator model

x=torch.randn((1,3,256,256))
model = Discriminator()
preds=model(x)
print(model)
print(preds.shape)

Discriminator(
  (initial): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
    (1): ReLU(inplace=True)
  )
  (model): Sequential(
    (0): Block(
      (conv): Sequential(
        (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): ReLU(inplace=True)
      )
    )
    (1): Block(
      (conv): Sequential(
        (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): ReLU(inplace=True)
      )
    )
    (2): Block(
      (conv): Sequential(
        (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running

In [ ]:
# generator model


class ConvBlock(nn.Module):
  def __init__(self,in_channels,out_channels,down=True,use_act=True,**kwargs):
    super().__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels,out_channels,padding_mode='reflect',**kwargs)
        if down
        else nn.ConvTranspose2d(in_channels,out_channels,**kwargs),
        nn.InstanceNorm2d(out_channels),
        nn.ReLU(inplace = True) if use_act else nn.Identity()
    )

  def forward(self,x):
    return self.conv(x)

In [ ]:
# generator model

class ResidualBlock(nn.Module):
  def __init__(self,channels):
    super().__init__()
    self.block = nn.Sequential(
        ConvBlock(channels,channels,kernel_size=3,padding=1),
        ConvBlock(channels,channels,use_act=False, kernel_size=3,padding=1),
    )

  def forward(self,x):
    return x + self.block(x)


In [ ]:
# generator model

import torch
import torch.nn as nn

class Generator(nn.Module):
  def __init__(self, img_channels, num_features=64, num_residuals=9):
      super(Generator, self).__init__()
      self.initial = nn.Sequential(
          nn.Conv2d(img_channels, num_features, kernel_size=7, stride=1, padding=3, padding_mode='reflect'),
          nn.ReLU(inplace=True)
      )

      self.down_blocks = nn.ModuleList([
          ConvBlock(num_features, num_features*2, kernel_size=3, stride=2, padding=1),
          ConvBlock(num_features*2, num_features*4, kernel_size=3, stride=2, padding=1)
      ])

      self.residual_blocks = nn.Sequential(*[ResidualBlock(num_features*4) for _ in range(num_residuals)])

      self.up_blocks = nn.ModuleList([
          ConvBlock(num_features*4, num_features*2, down=False, kernel_size=3, stride=2, padding=1, output_padding=1),
          ConvBlock(num_features*2, num_features, down=False, kernel_size=3, stride=2, padding=1, output_padding=1)
      ])

      self.last = nn.Conv2d(num_features, img_channels, kernel_size=7, stride=1, padding=3, padding_mode='reflect')

  def forward(self, x):
      x = self.initial(x)
      for layer in self.down_blocks:
          x = layer(x)
      x = self.residual_blocks(x)
      for layer in self.up_blocks:
          x = layer(x)
      return torch.tanh(self.last(x))


In [ ]:
# generator model

def test():
  img_channels = 3
  img_size = 256
  x=torch.randn((2,img_channels,img_size,img_size))
  gen=Generator(img_channels,9)
  print(gen)
  print(gen(x).shape)

In [ ]:
# generator model

img_channels = 3
img_size = 256
x=torch.randn((2,img_channels,img_size,img_size))
gen=Generator(img_channels,9)
print(gen)
print(gen(x).shape)

Generator(
  (initial): Sequential(
    (0): Conv2d(3, 9, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), padding_mode=reflect)
    (1): ReLU(inplace=True)
  )
  (down_blocks): ModuleList(
    (0): ConvBlock(
      (conv): Sequential(
        (0): Conv2d(9, 18, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(18, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): ReLU(inplace=True)
      )
    )
    (1): ConvBlock(
      (conv): Sequential(
        (0): Conv2d(18, 36, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(36, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): ReLU(inplace=True)
      )
    )
  )
  (residual_blocks): Sequential(
    (0): ResidualBlock(
      (block): Sequential(
        (0): ConvBlock(
          (conv): Sequential(
            (0): Conv2d(36, 36, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), pa

In [ ]:
# config

import albumentations as A
from albumentations.pytorch import ToTensorV2

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
TRAIN_DIR = 'data/train'
VAL_DIR = 'data/val'
BATCH_SIZE = 1
LEARNING_RATE = 2e-4
LAMBDA_IDENTITY = 0.0
LAMBDA_CYCLE = 10
NUM_WORKERS = 4
NUM_EPOCH = 200
LOAD_MODEL = True
SAVE_MODEL = True
CHECKPOINT_GEN_Z = 'genz.pth.tar'
CHECKPOINT_GEN_H = 'genh.pth.tar'
CHECKPOINT_CRITIC_H = 'critich.pth.tar'
CHECKPOINT_CRITIC_Z = 'critich.pth.tar'

transforms = A.Compose(
    [
        A.Resize(width=256,height=256),
        A.HorizontalFlip(p=0.5),
        A.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5],max_pixel_value=255),
        ToTensorV2(),
    ],
    additional_targets={'image0':'image'},
)

In [ ]:
# dataset

class HorseZebraDataset(Dataset):
  def __init__(self,root_zebra,root_horse,transform = None):
    self.root_zebra = root_zebra
    self.root_horse = root_horse
    self.transform = transform

    self.zebra_images = os.listdir(root_zebra)
    self.horse_images = os.listdir(root_horse)
    self.length_dataset = max (len(self.zebra_images),len(self.horse_images))
    self.zebra_len = len(self.zebra_images)
    self.horse_len = len(self.horse_images)

  def __len__(self):
    return self.length_dataset

  def __getitem__(self, index):
    zebra_img = self.zebra_images[index % self.zebra_len]
    horse_img = self.horse_images[index % self.horse_len]

    zebra_path = os.path.join(self.root_zebra,zebra_img)
    horse_path = os.path.join(self.root_horse,horse_img)

    zebra_img = np.array(Image.open(zebra_path).convert('RGB'))
    horse_img = np.array(Image.open(horse_path).convert('RGB'))

    if self.transform:
      augmentations = self.transform(image=zebra_img,image0=horse_img)
      zebra_img = augmentations['iamge']
      horse_img = augmentations['image0']

    return zebra_img,horse_img



In [ ]:
# utils

import random, torch, os, numpy as np
import torch.nn as nn
import copy

def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=DEVICE)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    # and it will lead to many hours of debugging \:
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr


def seed_everything(seed=42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
# training

import torch
import sys
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision.utils import save_image



def train_fn(
    disc_H, disc_Z, gen_Z, gen_H, loader, opt_disc, opt_gen, l1, mse, d_scaler, g_scaler):
    H_reals = 0
    H_fakes = 0
    loop = tqdm(loader, leave=True)

    for idx, (zebra, horse) in enumerate(loop):
        zebra = zebra.to(DEVICE)
        horse = horse.to(DEVICE)

        # Train Discriminators H and Z
        with torch.cuda.amp.autocast():
            fake_horse = gen_H(zebra)
            D_H_real = disc_H(horse)
            D_H_fake = disc_H(fake_horse.detach())
            H_reals += D_H_real.mean().item()
            H_fakes += D_H_fake.mean().item()
            D_H_real_loss = mse(D_H_real, torch.ones_like(D_H_real))
            D_H_fake_loss = mse(D_H_fake, torch.zeros_like(D_H_fake))
            D_H_loss = D_H_real_loss + D_H_fake_loss

            fake_zebra = gen_Z(horse)
            D_Z_real = disc_Z(zebra)
            D_Z_fake = disc_Z(fake_zebra.detach())
            D_Z_real_loss = mse(D_Z_real, torch.ones_like(D_Z_real))
            D_Z_fake_loss = mse(D_Z_fake, torch.zeros_like(D_Z_fake))
            D_Z_loss = D_Z_real_loss + D_Z_fake_loss

            # put it togethor
            D_loss = (D_H_loss + D_Z_loss) / 2

        opt_disc.zero_grad()
        d_scaler.scale(D_loss).backward()
        d_scaler.step(opt_disc)
        d_scaler.update()

        # Train Generators H and Z
        with torch.cuda.amp.autocast():
            # adversarial loss for both generators
            D_H_fake = disc_H(fake_horse)
            D_Z_fake = disc_Z(fake_zebra)
            loss_G_H = mse(D_H_fake, torch.ones_like(D_H_fake))
            loss_G_Z = mse(D_Z_fake, torch.ones_like(D_Z_fake))

            # cycle loss
            cycle_zebra = gen_Z(fake_horse)
            cycle_horse = gen_H(fake_zebra)
            cycle_zebra_loss = l1(zebra, cycle_zebra)
            cycle_horse_loss = l1(horse, cycle_horse)

            # identity loss
            identity_zebra = gen_Z(zebra)
            identity_horse = gen_H(horse)
            identity_zebra_loss = l1(zebra, identity_zebra)
            identity_horse_loss = l1(horse, identity_horse)

            # add all togethor
            G_loss = (
                loss_G_Z
                + loss_G_H
                + cycle_zebra_loss * LAMBDA_CYCLE
                + cycle_horse_loss * LAMBDA_CYCLE
                + identity_horse_loss * LAMBDA_IDENTITY
                + identity_zebra_loss * LAMBDA_IDENTITY
            )

        opt_gen.zero_grad()
        g_scaler.scale(G_loss).backward()
        g_scaler.step(opt_gen)
        g_scaler.update()

        if idx % 200 == 0:
            save_image(fake_horse * 0.5 + 0.5, f"saved_images/horse_{idx}.png")
            save_image(fake_zebra * 0.5 + 0.5, f"saved_images/zebra_{idx}.png")

        loop.set_postfix(H_real=H_reals / (idx + 1), H_fake=H_fakes / (idx + 1))


def main():
    disc_H = Discriminator(in_channels=3).to(DEVICE)
    disc_Z = Discriminator(in_channels=3).to(DEVICE)
    gen_Z = Generator(img_channels=3, num_residuals=9).to(DEVICE)
    gen_H = Generator(img_channels=3, num_residuals=9).to(DEVICE)
    opt_disc = optim.Adam(
        list(disc_H.parameters()) + list(disc_Z.parameters()),
        lr=LEARNING_RATE,
        betas=(0.5, 0.999),
    )

    opt_gen = optim.Adam(
        list(gen_Z.parameters()) + list(gen_H.parameters()),
        lr=LEARNING_RATE,
        betas=(0.5, 0.999),
    )

    L1 = nn.L1Loss()
    mse = nn.MSELoss()

    if LOAD_MODEL:
        load_checkpoint(
            CHECKPOINT_GEN_H,
            gen_H,
            opt_gen,
            LEARNING_RATE,
        )
        load_checkpoint(
            CHECKPOINT_GEN_Z,
            gen_Z,
            opt_gen,
            LEARNING_RATE,
        )
        load_checkpoint(
            CHECKPOINT_CRITIC_H,
            disc_H,
            opt_disc,
            LEARNING_RATE,
        )
        load_checkpoint(
            CHECKPOINT_CRITIC_Z,
            disc_Z,
            opt_disc,
            LEARNING_RATE,
        )

    dataset = HorseZebraDataset(
        root_horse=TRAIN_DIR + "/horses",
        root_zebra=TRAIN_DIR + "/zebras",
        transform=transforms,
    )
    val_dataset = HorseZebraDataset(
        root_horse="cyclegan_test/horse1",
        root_zebra="cyclegan_test/zebra1",
        transform=transforms,
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=1,
        shuffle=False,
        pin_memory=True,
    )
    loader = DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True,
    )
    g_scaler = torch.cuda.amp.GradScaler()
    d_scaler = torch.cuda.amp.GradScaler()

    for epoch in range(NUM_EPOCH):
        train_fn(
            disc_H,
            disc_Z,
            gen_Z,
            gen_H,
            loader,
            opt_disc,
            opt_gen,
            L1,
            mse,
            d_scaler,
            g_scaler,
        )

        if SAVE_MODEL:
            save_checkpoint(gen_H, opt_gen, filename=CHECKPOINT_GEN_H)
            save_checkpoint(gen_Z, opt_gen, filename=CHECKPOINT_GEN_Z)
            save_checkpoint(disc_H, opt_disc, filename=CHECKPOINT_CRITIC_H)
            save_checkpoint(disc_Z, opt_disc, filename=CHECKPOINT_CRITIC_Z)


if __name__ == "__main__":
    main()